# Gene Ontology Annotation in Python

(C) Andre O. Falcao (SI/DI/FCUL) 2018/2019

The purpose of this Notebook is to show you how to retrieve Gene Ontology (GO) annotation data for biological products, using simple Web data retrieval mechanisms.

We will use the **Uniprot** database for retrieving curated GO data from known proteins, and then **QuickGO** for getting their further information about the Gene Ontology terms (inlcuding name and ancestors).

## 1. Introduction

### Programmatic access to UniProt

During the prctical class, you studies the *Human Hemoglobin subunit beta*, Uniprot identifier `P68871`. For that, you accessed its UniProt record [http://www.uniprot.org/uniprot/P6887] from a web browser. In this exercise, we will access the data programmatically, which means that we need the data to be in a more structured format [http://www.uniprot.org/uniprot/P68871.txt].

### Python

In this classes, our tool of automation will be the Python programming language. Familiarity with this programming language is *highly* recommended, although the tutorial can be followed without it. Please learn the basics of Python on your own, if you have not done so yet.

## 2. Getting data from UniProt

We can read the full UniProt record of the protein `P68871` using Python, taking advantage of its built in `requests` library

In [3]:
import requests

response = requests.get('http://www.uniprot.org/uniprot/P68871.txt')
data = response.text
data_lines = data.splitlines()

To print out the complete record, we just need to go through all the lines read

In [4]:
for line in data_lines:
    print(line)

ID   HBB_HUMAN               Reviewed;         147 AA.
AC   P68871; A4GX73; B2ZUE0; P02023; Q13852; Q14481; Q14510; Q45KT0;
AC   Q549N7; Q6FI08; Q6R7N2; Q8IZI1; Q9BX96; Q9UCD6; Q9UCP8; Q9UCP9;
DT   21-JUL-1986, integrated into UniProtKB/Swiss-Prot.
DT   23-JAN-2007, sequence version 2.
DT   16-JAN-2019, entry version 178.
DE   RecName: Full=Hemoglobin subunit beta;
DE   AltName: Full=Beta-globin;
DE   AltName: Full=Hemoglobin beta chain;
DE   Contains:
DE     RecName: Full=LVV-hemorphin-7;
DE   Contains:
DE     RecName: Full=Spinorphin;
GN   Name=HBB;
OS   Homo sapiens (Human).
OC   Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
OC   Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
OC   Catarrhini; Hominidae; Homo.
OX   NCBI_TaxID=9606;
RN   [1]
RP   NUCLEOTIDE SEQUENCE [GENOMIC DNA].
RX   PubMed=1019344;
RA   Marotta C., Forget B., Cohen-Solal M., Weissman S.M.;
RT   "Nucleotide sequence analysis of coding and noncoding regions of human
RT   beta-globin 

Each line contains a specific type of information. The type is specified with the first two characters of the line. For instance, a line that starts with `OS` has information about the organism where the protein is expressed.

(Full list of UniProt line types here: [https://www.uniprot.org/docs/userman#linetypes])

In [7]:
for line in data_lines:
    if line.startswith('OS'):
        print(line)

OS   Homo sapiens (Human).


### 2.1 Getting GO Terms from UniProt

The `DR` (database Cross-Reference) line type has a multitude of sub types, so we need to identify which lines correspond to the GO terms. By looking closely into the record, we identify that we need to find lines that start with the 7 characters `DR   GO`:

In each of these lines, there are several fields of data separated by semi-colons (`;`). We can process and extract the relevant information, namely the GO identifier, the term name and the evidence code associated with each term:

In [8]:
GO_terms = {}

for line in data_lines:
    if line.startswith('DR   GO'):
        line_prefix, go_id, go_term_full, evidence_code = line.split(';')
        go_id = go_id.strip()
        go_term_full = go_term_full.strip()
        evidence_code = evidence_code.strip()
        go_domain, go_term_name = go_term_full.split(':')
        
        GO_terms[go_id] = (go_domain, go_term_name, evidence_code)

We can now have a cleaner view of the GO data retrieved from the UniProt record

In [9]:
for go_id in GO_terms:
    go_info = GO_terms[go_id]
    
    print(go_id)
    print('\tGO Domain:', go_info[0])
    print('\tGO Term:', go_info[1])
    print('\tEvidence code:', go_info[2])

GO:0072562
	GO Domain: C
	GO Term: blood microparticle
	Evidence code: HDA:UniProtKB.
GO:0005829
	GO Domain: C
	GO Term: cytosol
	Evidence code: TAS:Reactome.
GO:0071682
	GO Domain: C
	GO Term: endocytic vesicle lumen
	Evidence code: TAS:Reactome.
GO:0070062
	GO Domain: C
	GO Term: extracellular exosome
	Evidence code: HDA:UniProtKB.
GO:0005576
	GO Domain: C
	GO Term: extracellular region
	Evidence code: TAS:Reactome.
GO:0005615
	GO Domain: C
	GO Term: extracellular space
	Evidence code: IDA:UniProtKB.
GO:1904813
	GO Domain: C
	GO Term: ficolin-1-rich granule lumen
	Evidence code: TAS:Reactome.
GO:0031838
	GO Domain: C
	GO Term: haptoglobin-hemoglobin complex
	Evidence code: IDA:BHF-UCL.
GO:0005833
	GO Domain: C
	GO Term: hemoglobin complex
	Evidence code: IDA:BHF-UCL.
GO:1904724
	GO Domain: C
	GO Term: tertiary granule lumen
	Evidence code: TAS:Reactome.
GO:0020037
	GO Domain: F
	GO Term: heme binding
	Evidence code: IBA:GO_Central.
GO:0031721
	GO Domain: F
	GO Term: hemoglobin alpha 

## 3. Improving the GO retrieval process using QuickGO

The above procedure is quite useful but shows only a fraction of all the terms associated with each protein. This happens because all of the ancestors of a term (the GO terms that appear "above" the GO tree and with which they have a "is_a" relationship) are not shown.

A nice little way to supplement the above information is to access the [QuickGO database](https://www.ebi.ac.uk/QuickGO/) to access that information.

This will be slightly more complex than the above procedure, as the data in QuickGO can only be accessed as a RESTful Web Service [https://en.wikipedia.org/wiki/Representational_state_transfer]. This means that a lot more power is given to users to access the data and we do not need to parse web pages to retrieve it. We just need to build a "query URL" that is able to specify the data that we require, and by accessing that URL, data will be retrieved, already formated for programtaic consumption. Typically the results of a REST query are formated as JSON or XML (Google "JSON" or "XML" if you are at a loss and REALLY want to know more!).

Python already comes equiped to deal with webservices, XML and JSON, and so you do not need to understand those technicalities in depth. The only thing that we need to take care is to know which types of request can be asked to the QuickGO webservice, how to build the URL query to ask for those requests, and how the QuickGO webservice retrieves the results (in particular, how the ancestors of a term are retrieved). 

The QuickGO Web API documentation can be found here: https://www.ebi.ac.uk/QuickGO/api/index.html

Please note that we will only retrieve the "is_a" parents, although other types of relationships could be requested as well, providing all types of relationship separated by commas (`,`). See the above link for the full specification.

We are going to need the `json` library to deal with the JSON response given by the webservice.

### 3.1 An example with a single GO term

Let's start small: let's ask the webservice to give us the ancestors of the term `GO:0072562`.

In [10]:
import json

the_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/GO:0072562/ancestors?relations=is_a'

response = requests.get(the_url)
anc_data = json.loads(response.text)

The keys of `anc_data` (the response returned by the webservice) are:

In [11]:
for key in anc_data.keys():
    print(key)

numberOfHits
results
pageInfo


Only the `results` item actually matters for our use case, as it contains the actual ancestors of our term:

In [12]:
results = anc_data['results']
print(results)

[{'id': 'GO:0072562', 'isObsolete': False, 'name': 'blood microparticle', 'definition': {'text': 'A phospholipid microvesicle that is derived from any of several cell types, such as platelets, blood cells, endothelial cells, or others, and contains membrane receptors as well as other proteins characteristic of the parental cell. Microparticles are heterogeneous in size, and are characterized as microvesicles free of nucleic acids.', 'xrefs': [{'dbCode': 'PMID', 'dbId': '16373184'}]}, 'ancestors': ['GO:0072562', 'GO:0044421', 'GO:0005575'], 'synonyms': [{'name': 'cell membrane microparticle', 'type': 'exact'}], 'children': [{'id': 'GO:0072563', 'relation': 'is_a'}], 'aspect': 'cellular_component', 'usage': 'Unrestricted'}]


Fortunately Python makes sorting through that mess a breeze:

In [13]:
for record in results:
    print('>>>', record['id'])
    print('\tName:', record['name'])
    print('\tDefinition:', record['definition']['text'])
    print('\tAncestors:', record['ancestors'])

>>> GO:0072562
	Name: blood microparticle
	Definition: A phospholipid microvesicle that is derived from any of several cell types, such as platelets, blood cells, endothelial cells, or others, and contains membrane receptors as well as other proteins characteristic of the parental cell. Microparticles are heterogeneous in size, and are characterized as microvesicles free of nucleic acids.
	Ancestors: ['GO:0072562', 'GO:0044421', 'GO:0005575']


### 3.2 Getting the ancestors of multiple terms

Section 2 shows us how to extract the GO annotation from a UniProt entry; Section 3.1 shows us how to retrieve the ancestors for one term. What we want now is to automate the whole process: getting all the ancestors for all the terms of one protein.

We can start by noticing that the above request can work for multiple GO identifiers simultaneously. We just need to separate the identifier with a comma (`,`), so we can build a URL with all the identified GO terms from a single UniProt entry.

Remember that we have already created the list `go_ids` which contains the GO terms that annotate the protein `P68871`.

In [14]:
go_ids = GO_terms.keys()
the_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/'
the_url += ','.join(go_ids)
the_url += '/ancestors?relations=is_a'
print(the_url)

https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/GO:0072562,GO:0005829,GO:0071682,GO:0070062,GO:0005576,GO:0005615,GO:1904813,GO:0031838,GO:0005833,GO:1904724,GO:0020037,GO:0031721,GO:0030492,GO:0046872,GO:0043177,GO:0019825,GO:0005344,GO:0015701,GO:0007596,GO:0098869,GO:0042744,GO:0043312,GO:0030185,GO:0015671,GO:0070527,GO:0010942,GO:0045429,GO:0051291,GO:0006898,GO:0008217,GO:0050880,GO:0070293,GO:0042542/ancestors?relations=is_a


As an exercise, try to open that link in the browser to see the raw results provided by the webservice.

Right, now we need to instruct python to make the request to that url and print the results in a easy to read format

In [15]:
response = requests.get(the_url)
anc_data = json.loads(response.text)
results = anc_data['results']
distinct_terms = set()

for record in results:
    print('>>>', record['id'])
    print('\tName:', record['name'])
    print('\tDefinition:', record['definition']['text'])
    print('\tAncestors:', record['ancestors'])
    
    # Add all the ancestors returned in this record to the global set of ancestors
    distinct_terms.update(record['ancestors'])

>>> GO:0030185
	Name: nitric oxide transport
	Definition: The directed movement of nitric oxide, nitrogen monoxide, into, out of or within a cell, or between cells, by means of some agent such as a transporter or pore.
	Ancestors: ['GO:0006836', 'GO:0008150', 'GO:0071705', 'GO:0051179', 'GO:0051234', 'GO:0015893', 'GO:0030185', 'GO:0006810']
>>> GO:0042744
	Name: hydrogen peroxide catabolic process
	Definition: The chemical reactions and pathways resulting in the breakdown of hydrogen peroxide (H2O2).
	Ancestors: ['GO:0008150', 'GO:0016999', 'GO:0042737', 'GO:0009056', 'GO:0008152', 'GO:0044237', 'GO:0044248', 'GO:0042743', 'GO:0009987', 'GO:0042744', 'GO:0072593', 'GO:0051187', 'GO:0051186', 'GO:0017144', 'GO:0017001']
>>> GO:0015671
	Name: oxygen transport
	Definition: The directed movement of oxygen (O2) into, out of or within a cell, or between cells, by means of some agent such as a transporter or pore.
	Ancestors: ['GO:0008150', 'GO:0015893', 'GO:0051234', 'GO:0051179', 'GO:00156

To have a cleaner view of all the distinct terms identified:

In [16]:
print('The number of Distinct terms is', len(distinct_terms))
print()
print(distinct_terms)

The number of Distinct terms is 163

{'GO:0002366', 'GO:0046872', 'GO:0051173', 'GO:0031328', 'GO:0003013', 'GO:0003014', 'GO:0017001', 'GO:0044446', 'GO:1904724', 'GO:0008144', 'GO:0045055', 'GO:0006811', 'GO:0051234', 'GO:1904407', 'GO:0044445', 'GO:0043227', 'GO:0044444', 'GO:0032991', 'GO:0031982', 'GO:0046903', 'GO:0070013', 'GO:0044433', 'GO:0046906', 'GO:0097159', 'GO:0044424', 'GO:0001775', 'GO:0031838', 'GO:0050794', 'GO:0009891', 'GO:0050878', 'GO:0006887', 'GO:0043167', 'GO:0006979', 'GO:0016043', 'GO:1990748', 'GO:0044248', 'GO:0019222', 'GO:0042542', 'GO:0002376', 'GO:0098657', 'GO:0046677', 'GO:0050896', 'GO:0015711', 'GO:0042119', 'GO:0034109', 'GO:1903561', 'GO:0043177', 'GO:0090066', 'GO:1903428', 'GO:0031326', 'GO:0098609', 'GO:0048518', 'GO:0006810', 'GO:0044464', 'GO:0043312', 'GO:0070293', 'GO:0015701', 'GO:0006836', 'GO:0070062', 'GO:0010941', 'GO:0071840', 'GO:0005615', 'GO:1901363', 'GO:0010035', 'GO:0032501', 'GO:0005829', 'GO:0042743', 'GO:0009987', 'GO:005118

## 4. Putting it all together

Now, let’s wrap it up in a function that receives a UniProt identifer and returns all the GO identifiers associated with that protein, including ancestors.

In [17]:
import json
import requests

def get_full_annotation(uniprot_id):
    distinct_terms = set()
    
    # First get the Uniprot data
    response = requests.get('http://www.uniprot.org/uniprot/' + uniprot_id + '.txt')
    data = response.text
    data_lines = data.splitlines()
    
    # Look for the GO identifiers in the response and save them in a set
    go_ids = set()
    for line in data_lines:
        if line.startswith('DR   GO'):
            line_prefix, go_id, go_term_full, evidence_code = line.split(';')
            go_id = go_id.strip()
            go_ids.add(go_id)
    
    # Construct the URL for the QuickGO Request with all those GO identifiers
    the_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/'
    the_url += ','.join(go_ids)
    the_url += '/ancestors?relations=is_a'
    
    # Make the request and parse the results
    response = requests.get(the_url)
    anc_data = json.loads(response.text)
    results = anc_data['results']
    for record in results:
        distinct_terms.update(record['ancestors'])
                                        
    return distinct_terms

Does it work?

In [18]:
go_ids = get_full_annotation('P69905')
print(go_ids)

{'GO:0046872', 'GO:0016020', 'GO:0017001', 'GO:0044446', 'GO:0008144', 'GO:0006811', 'GO:0051234', 'GO:0044445', 'GO:0043227', 'GO:0070013', 'GO:0044444', 'GO:0032991', 'GO:0031982', 'GO:0044433', 'GO:0044391', 'GO:0046906', 'GO:0097159', 'GO:0044424', 'GO:0031838', 'GO:0005506', 'GO:0050794', 'GO:0043167', 'GO:0006979', 'GO:0016043', 'GO:1990748', 'GO:0044248', 'GO:0042542', 'GO:0098657', 'GO:0046677', 'GO:0050896', 'GO:0015711', 'GO:1903561', 'GO:0043177', 'GO:0015935', 'GO:1990904', 'GO:0048518', 'GO:0006810', 'GO:0044464', 'GO:0015701', 'GO:0070062', 'GO:0010941', 'GO:0071840', 'GO:0005615', 'GO:1901363', 'GO:0010035', 'GO:0005829', 'GO:0042743', 'GO:0009987', 'GO:0051187', 'GO:0051291', 'GO:0044422', 'GO:0065007', 'GO:0019825', 'GO:0016192', 'GO:0042221', 'GO:0072562', 'GO:0043230', 'GO:0006897', 'GO:0043933', 'GO:0071702', 'GO:0065003', 'GO:0020037', 'GO:0022627', 'GO:0072593', 'GO:0006950', 'GO:0048522', 'GO:0043169', 'GO:0006898', 'GO:0015671', 'GO:0022607', 'GO:0015669', 'GO:0

This function returns a collection of GO terms. However, it does not provide any insight into what they actually represent. At least, we wnat to know the name of the term and the GO branch where they belong to (molecular function, biological process or cellular component). For that, we can use again the QuickGO API.

To that effect we will create function that takes a GO identifier and returns its information. As an exercise, we extract the name of the GO term, its GO branch (or aspect), the textual definition and whether that term is obsolete.

In [19]:
def get_term_info(go_id):
    the_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/' + go_id
    response = requests.get(the_url)
    data = json.loads(response.text)
    
    record = data['results'][0] # Get the first result, because we are only requesting one!
    
    # Return a dictionary that contains the information of this GO term
    go_info = {
        'go_id': record['id'],
        'aspect': record['aspect'],
        'name': record['name'],
        'definition': record['definition']['text'],
        'obsolete': record['isObsolete'],
    }
    
    return go_info

Here is a demonstration of the function in action:

In [20]:
go_info = get_term_info('GO:0046872')

print('GO ID:', go_info['go_id'])
print('\tTerm name:', go_info['name'])
print('\tGO Domain:', go_info['aspect'])
print('\tdefinition:', go_info['definition'])
print('\tIs term Obsolete?', go_info['obsolete'])

GO ID: GO:0046872
	Term name: metal ion binding
	GO Domain: molecular_function
	definition: Interacting selectively and non-covalently with any metal ion.
	Is term Obsolete? False


## 5. GO term counting

The above functions have so far been used for information of a single protein. Yet, it should be easy to use them in a more complex problem where we might want to identify the most common terms in a **set of proteins**. Let's consider the trhee following proteins: lobster haemocyanin (`P10787`), human hemoglobin subunit beta (`P68871`), human myoglobin (`P02144`).

In [20]:
proteins = ['P10787', 'P68871', 'P02144']

all_terms = []
for protein_id in proteins:
    all_terms.extend(get_full_annotation(protein_id))

term_counts = {term: all_terms.count(term) for term in all_terms}
distinct_terms = list(term_counts)

# Examine the first 10 terms of the term_counts
for go_id in distinct_terms[:10]:
    name = get_term_info(go_id)['name']
    print(go_id, '// count:', term_counts[go_id], '// Name:', name)

GO:0003674 // count: 3 // Name: molecular_function
GO:0140104 // count: 3 // Name: molecular carrier activity
GO:0044421 // count: 3 // Name: extracellular region part
GO:0046872 // count: 3 // Name: metal ion binding
GO:0005488 // count: 3 // Name: binding
GO:0043167 // count: 3 // Name: ion binding
GO:0005575 // count: 3 // Name: cellular_component
GO:0005615 // count: 2 // Name: extracellular space
GO:0003824 // count: 1 // Name: catalytic activity
GO:0005344 // count: 3 // Name: oxygen carrier activity


Which terms appear in all three proteins? (as we have 3 proteins, we are looking for the terms that appear 3 times)

We will use the function `get_term_info` to print out the most relevant information for each GO term retrieved previously.

In [21]:
for go_id in distinct_terms:
    if term_counts[go_id] == 3:
        go_info = get_term_info(go_id)
        print(go_id, '// Apect:', go_info['aspect'], '// Name:', go_info['name'])

GO:0003674 // Apect: molecular_function // Name: molecular_function
GO:0140104 // Apect: molecular_function // Name: molecular carrier activity
GO:0044421 // Apect: cellular_component // Name: extracellular region part
GO:0046872 // Apect: molecular_function // Name: metal ion binding
GO:0005488 // Apect: molecular_function // Name: binding
GO:0043167 // Apect: molecular_function // Name: ion binding
GO:0005575 // Apect: cellular_component // Name: cellular_component
GO:0005344 // Apect: molecular_function // Name: oxygen carrier activity
GO:0043169 // Apect: molecular_function // Name: cation binding
